<a href="https://colab.research.google.com/github/chechae/mywork/blob/main/try10_5_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. pre

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install category_encoders
#운영체제 및 파일 읽기
import os
import glob
#기초 데이터 처리
import pandas as pd
import numpy as np

#변수 인코딩
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import category_encoders as ce
from category_encoders import OrdinalEncoder,TargetEncoder, GLMMEncoder

#오버샘플링
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

#머신러닝 패키지
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
#+추가
from sklearn import metrics #accuracy measure
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
import graphviz
from sklearn import svm #support vector Machine
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from xgboost import XGBClassifier #XGBoost
import lightgbm as lgb #LightGBM
from sklearn import tree

#시각화
import matplotlib.pyplot as plt
import seaborn as sns

#기타 제반 사항
from google.colab import output
import warnings



#출력창을 더 깔끔하게 하기 위함
warnings.filterwarnings("ignore")
output.clear()

In [3]:
encoders = {
    'BinaryEncoder': ce.binary.BinaryEncoder,
    'CatBoostEncoder': ce.cat_boost.CatBoostEncoder,
    'HashingEncoder': ce.hashing.HashingEncoder,
    'HelmertEncoder': ce.helmert.HelmertEncoder,
    'JamesSteinEncoder': ce.james_stein.JamesSteinEncoder,
    'OneHotEncoder': ce.one_hot.OneHotEncoder,
    'LeaveOneOutEncoder': ce.leave_one_out.LeaveOneOutEncoder,
    'MEstimateEncoder': ce.m_estimate.MEstimateEncoder,
    'OrdinalEncoder': ce.ordinal.OrdinalEncoder,
    'PolynomialEncoder': ce.polynomial.PolynomialEncoder,
    'SumEncoder': ce.sum_coding.SumEncoder,
    'TargetEncoder': ce.target_encoder.TargetEncoder,
    'WOEEncoder': ce.woe.WOEEncoder
}

In [10]:
path_train = '/content/drive/MyDrive/24_1_pattern_recognition/data/train1.csv'
path_test = '/content/drive/MyDrive/24_1_pattern_recognition/data/test1.csv'
df_test = pd.read_csv(path_test)
df_train = pd.read_csv(path_train) ; df_train.head()

,id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,12224,46,admin.,married,high.school,unknown,no,no,telephone,jul,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.966,5228.1,no
1,36387,32,admin.,married,university.degree,no,yes,yes,cellular,jun,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2,yes
2,22703,36,technician,single,university.degree,no,no,no,cellular,aug,...,5,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,no
3,32742,30,services,single,high.school,no,yes,yes,cellular,may,...,1,999,2,failure,-1.8,92.893,-46.2,1.299,5099.1,no
4,40547,76,retired,divorced,basic.4y,no,no,no,cellular,aug,...,1,999,1,failure,-1.7,94.027,-38.3,0.886,4991.6,yes


In [11]:
num_cols_list = df_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
num_cols_list

['id',
 'age',
 'campaign',
 'pdays',
 'previous',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed']

In [12]:
cat_list = df_train.select_dtypes(include=['object']).columns.tolist()
cat_list

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome',
 'y']

## 1. Preprocessing

### 1-1. Data cleaning

#### 1-1. Missing values

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 데이터 불러오기
df_train = pd.read_csv(path_train)

# 수치형 데이터에 대한 결측치 처리: 반복적 임퓨터 사용
numeric_columns = df_train.select_dtypes(include=[np.number]).columns
imp_numeric = IterativeImputer(max_iter=10, random_state=0)
df_train[numeric_columns] = imp_numeric.fit_transform(df_train[numeric_columns])

# 범주형 데이터에 대한 결측치 처리: 최빈값 사용
categorical_columns = df_train.select_dtypes(include=['object']).columns
imp_categorical = SimpleImputer(strategy='most_frequent')
df_train[categorical_columns] = imp_categorical.fit_transform(df_train[categorical_columns])

# 결과 확인
print(df_train.head())

In [16]:
import pandas as pd
import numpy as np

# 데이터 로드
df_train = pd.read_csv(path_train)

# 'unknown' 값을 NaN으로 대체
df_train.replace('unknown', np.nan, inplace=True)

# SimpleImputer 또는 다른 방법을 사용하여 결측치 처리
from sklearn.impute import SimpleImputer

# 범주형 데이터를 위한 임퓨터 생성, 여기서는 예를 들어 최빈값으로 채우는 것을 선택
imputer = SimpleImputer(strategy='most_frequent')

# 범주형 열만 선택
categorical_columns = df_train.select_dtypes(include=['object']).columns
df_train[categorical_columns] = imputer.fit_transform(df_train[categorical_columns])

# 결과 확인
print(df_train.head())

      id  age         job   marital          education default housing loan  \
0  12224   46      admin.   married        high.school      no      no   no   
1  36387   32      admin.   married  university.degree      no     yes  yes   
2  22703   36  technician    single  university.degree      no      no   no   
3  32742   30    services    single        high.school      no     yes  yes   
4  40547   76     retired  divorced           basic.4y      no      no   no   

     contact month  ... campaign  pdays  previous     poutcome emp.var.rate  \
0  telephone   jul  ...        3    999         0  nonexistent          1.4   
1   cellular   jun  ...        1    999         0  nonexistent         -2.9   
2   cellular   aug  ...        5    999         0  nonexistent          1.4   
3   cellular   may  ...        1    999         2      failure         -1.8   
4   cellular   aug  ...        1    999         1      failure         -1.7   

   cons.price.idx  cons.conf.idx  euribor3m  nr.em

In [17]:
df_train.head()

,id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,12224,46,admin.,married,high.school,no,no,no,telephone,jul,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.966,5228.1,no
1,36387,32,admin.,married,university.degree,no,yes,yes,cellular,jun,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2,yes
2,22703,36,technician,single,university.degree,no,no,no,cellular,aug,...,5,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,no
3,32742,30,services,single,high.school,no,yes,yes,cellular,may,...,1,999,2,failure,-1.8,92.893,-46.2,1.299,5099.1,no
4,40547,76,retired,divorced,basic.4y,no,no,no,cellular,aug,...,1,999,1,failure,-1.7,94.027,-38.3,0.886,4991.6,yes


In [18]:
# 이상치를 식별하고 카운트하는 함수
def find_outliers(df):
    outlier_indices = []
    for column in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        outlier_list_col = df[(df[column] < Q1 - 1.5 * IQR) | (df[column] > Q3 + 1.5 * IQR)].index
        outlier_indices.extend(outlier_list_col)

    outlier_indices = list(set(outlier_indices))  # 중복 인덱스 제거
    return outlier_indices

# 각 컬럼의 이상치 개수 계산
outlier_counts = {}
for column in df_train.select_dtypes(include=[np.number]).columns:
    Q1 = df_train[column].quantile(0.25)
    Q3 = df_train[column].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_train[(df_train[column] < Q1 - 1.5 * IQR) | (df_train[column] > Q3 + 1.5 * IQR)]
    outlier_counts[column] = outliers.shape[0]

# 이상치 수 출력
for column, count in outlier_counts.items():
    print(f"{column}: {count} outliers")

# 이상치가 가장 많은 컬럼 찾기
max_outliers = max(outlier_counts, key=outlier_counts.get)
print(f"\nThe column with the most outliers is {max_outliers} with {outlier_counts[max_outliers]} outliers.")

id: 0 outliers
age: 380 outliers
campaign: 1920 outliers
pdays: 1214 outliers
previous: 4476 outliers
emp.var.rate: 0 outliers
cons.price.idx: 0 outliers
cons.conf.idx: 353 outliers
euribor3m: 0 outliers
nr.employed: 0 outliers

The column with the most outliers is previous with 4476 outliers.


<이상치 처리 방법>
- 영향 높(이상치 유지): cons.price.idx / poutcome / previous / emp.var.rate /
- 영향 중간: campaign / age (유지 or 최빈값이나 중앙값으로 대체
- 영향 낮: cons.conf.idx

In [19]:
# 'cons.conf.idx' 열의 중앙값 계산
median_value = df_train['cons.conf.idx'].median()

# IQR을 사용하여 이상치를 식별
Q1 = df_train['cons.conf.idx'].quantile(0.25)
Q3 = df_train['cons.conf.idx'].quantile(0.75)
IQR = Q3 - Q1

# 이상치의 하한과 상한 계산
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치를 중앙값으로 대체
df_train['cons.conf.idx'] = df_train['cons.conf.idx'].apply(
    lambda x: median_value if x < lower_bound or x > upper_bound else x
)

# 변경된 데이터 확인
print(df_train['cons.conf.idx'])

# 변경된 데이터를 파일로 저장 (필요한 경우)
df_train.to_csv('updated_data.csv', index=False)

0       -42.7
1       -40.8
2       -36.1
3       -46.2
4       -38.3
         ... 
32945   -42.0
32946   -42.7
32947   -42.7
32948   -41.8
32949   -36.1
Name: cons.conf.idx, Length: 32950, dtype: float64


In [20]:
# 'cons.conf.idx' 열의 중앙값 계산
median_value = df_train['cons.conf.idx'].median()

# IQR을 사용하여 이상치를 식별
Q1 = df_train['cons.conf.idx'].quantile(0.25)
Q3 = df_train['cons.conf.idx'].quantile(0.75)
IQR = Q3 - Q1

# 이상치의 하한과 상한 계산
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치를 중앙값으로 대체
df_train['cons.conf.idx'] = df_train['cons.conf.idx'].apply(
    lambda x: median_value if x < lower_bound or x > upper_bound else x
)

# 변경된 데이터 확인
print(df_train['cons.conf.idx'])

# 변경된 데이터를 파일로 저장 (필요한 경우)
df_train.to_csv('updated_data.csv', index=False)

0       -42.7
1       -40.8
2       -36.1
3       -46.2
4       -38.3
         ... 
32945   -42.0
32946   -42.7
32947   -42.7
32948   -41.8
32949   -36.1
Name: cons.conf.idx, Length: 32950, dtype: float64


In [22]:
# object 타입 category 타입으로 변환 (메모리 효율성 높임)
categorical_columns = df_train.select_dtypes(include=['object']).columns
df_train[categorical_columns] = df_train[categorical_columns].astype('category')

print(df_train.dtypes)

id                   int64
age                  int64
job               category
marital           category
education         category
default           category
housing           category
loan              category
contact           category
month             category
day_of_week       category
campaign             int64
pdays                int64
previous             int64
poutcome          category
emp.var.rate       float64
cons.price.idx     float64
cons.conf.idx      float64
euribor3m          float64
nr.employed        float64
y                 category
dtype: object


- one-hot encoding: job, marital, education, month, day_of_week, poutcome
- binary encoding: default, housing, loan, contact

In [23]:
# 원-핫 인코딩 적용
df_train_encoded = pd.get_dummies(df_train, columns=[
    'job', 'marital', 'education', 'month', 'day_of_week', 'poutcome'
])

# 인코딩 결과 확인
print(df_train_encoded.head())

      id  age default housing loan    contact  campaign  pdays  previous  \
0  12224   46      no      no   no  telephone         3    999         0   
1  36387   32      no     yes  yes   cellular         1    999         0   
2  22703   36      no      no   no   cellular         5    999         0   
3  32742   30      no     yes  yes   cellular         1    999         2   
4  40547   76      no      no   no   cellular         1    999         1   

   emp.var.rate  ...  month_oct  month_sep  day_of_week_fri  day_of_week_mon  \
0           1.4  ...      False      False            False            False   
1          -2.9  ...      False      False            False            False   
2           1.4  ...      False      False             True            False   
3          -1.8  ...      False      False            False             True   
4          -1.7  ...      False      False            False            False   

  day_of_week_thu  day_of_week_tue  day_of_week_wed  poutcome_

In [25]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 바이너리 인코딩을 적용할 범주형 변수 선택
columns_to_encode = ['default', 'housing', 'loan', 'contact']

# LabelEncoder를 사용하여 각 범주형 값을 숫자로 일시적 변환
label_encoders = {}
for column in columns_to_encode:
    le = LabelEncoder()
    df_train_encoded[column] = le.fit_transform(df_train_encoded[column])
    label_encoders[column] = le  # 나중에 사용할 수 있도록 저장

# 각 숫자 값을 이진 코드로 변환
for column in columns_to_encode:
    # 최대 길이 계산
    max_bits = int(df_train_encoded[column].max()).bit_length()
    # 각 범주를 이진 코드로 변환
    df_train_encoded[[f'{column}_bit_{i}' for i in range(max_bits)]] = df_train_encoded[column].apply(
        lambda x: pd.Series(list(bin(x)[2:].zfill(max_bits))).astype(int)
    )

    # 원래의 컬럼 삭제
    df_train_encoded.drop(column, axis=1, inplace=True)

# 결과 확인
print(df_train_encoded.head())

      id  age  campaign  pdays  previous  emp.var.rate  cons.price.idx  \
0  12224   46         3    999         0           1.4          93.918   
1  36387   32         1    999         0          -2.9          92.963   
2  22703   36         5    999         0           1.4          93.444   
3  32742   30         1    999         2          -1.8          92.893   
4  40547   76         1    999         1          -1.7          94.027   

   cons.conf.idx  euribor3m  nr.employed  ... day_of_week_thu  \
0          -42.7      4.966       5228.1  ...            True   
1          -40.8      1.262       5076.2  ...           False   
2          -36.1      4.964       5228.1  ...           False   
3          -46.2      1.299       5099.1  ...           False   
4          -38.3      0.886       4991.6  ...           False   

   day_of_week_tue  day_of_week_wed  poutcome_failure  poutcome_nonexistent  \
0            False            False             False                  True   
1     

In [26]:
# df_train_encoded 데이터프레임을 train_data로 이름 변경
train_data = df_train_encoded

In [27]:
# train_data에서 범주형 변수 확인
categorical_variables = train_data.select_dtypes(include=['object', 'category']).columns

# 범주형 변수 출력
if len(categorical_variables) > 0:
    print("다음은 추가 인코딩이 필요한 범주형 변수입니다:", list(categorical_variables))
else:
    print("더 이상 인코딩할 범주형 변수가 없습니다.")

다음은 추가 인코딩이 필요한 범주형 변수입니다: ['y']


In [28]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
le = LabelEncoder()

# `y` 열 인코딩
train_data['y'] = le.fit_transform(train_data['y'])

# 인코딩 결과 확인
print(train_data['y'].head())

0    0
1    1
2    0
3    0
4    1
Name: y, dtype: int64


In [31]:
df_train = train_data

# 모델링

In [32]:
X = df_train.drop('y', axis=1)
y = df_train['y']

# 데이터 분할: 훈련 데이터와 검증 데이터
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.1 MB/s eta 0:00:00


In [34]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# 데이터 분할
X = train_data.drop('y', axis=1)
y = train_data['y']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# CatBoost 모델 초기화
catboost_model = CatBoostClassifier(
    iterations=1000,  # 반복 횟수
    learning_rate=0.1,  # 학습률
    depth=6,  # 트리 깊이
    loss_function='Logloss',  # 손실 함수
    verbose=200  # 학습 과정 중 출력 빈도
)

# 모델 훈련
catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

# 검증 데이터에 대한 예측 수행
y_val_pred = catboost_model.predict(X_val)

# F1 score 계산
f1 = f1_score(y_val, y_val_pred)
print(f'Validation F1 Score: {f1:.4f}')

0:	learn: 0.5940764	test: 0.5939152	best: 0.5939152 (0)	total: 119ms	remaining: 1m 58s
200:	learn: 0.2423407	test: 0.2704386	best: 0.2693913 (107)	total: 6.02s	remaining: 23.9s
400:	learn: 0.2204320	test: 0.2745032	best: 0.2693913 (107)	total: 10.5s	remaining: 15.7s
600:	learn: 0.2037484	test: 0.2778532	best: 0.2693913 (107)	total: 12.4s	remaining: 8.25s
800:	learn: 0.1902899	test: 0.2819627	best: 0.2693913 (107)	total: 16.8s	remaining: 4.17s
999:	learn: 0.1780687	test: 0.2848674	best: 0.2693913 (107)	total: 19.2s	remaining: 0us

bestTest = 0.2693912971
bestIteration = 107

Shrink model to first 108 iterations.
Validation F1 Score: 0.4000


In [38]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# 데이터 준비
X = train_data.drop('y', axis=1)
y = train_data['y']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 초기화
catboost_model = CatBoostClassifier(
    iterations=2000, # 반복 횟수 증가
    learning_rate=0.05, # 학습률 감소
    depth=8, # 트리 깊이 조정
    loss_function='Logloss',
    verbose=200,
    scale_pos_weight=2 # 클래스 불균형 처리
)

# 모델 훈련
catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

# 예측 및 평가
y_val_pred = catboost_model.predict(X_val)
f1 = f1_score(y_val, y_val_pred)
print(f'Validation F1 Score: {f1:.4f}')

0:	learn: 0.6588240	test: 0.6589615	best: 0.6589615 (0)	total: 39.2ms	remaining: 1m 18s
200:	learn: 0.3386596	test: 0.3821412	best: 0.3807404 (115)	total: 8.11s	remaining: 1m 12s
400:	learn: 0.2922788	test: 0.3866131	best: 0.3807404 (115)	total: 12.1s	remaining: 48.4s
600:	learn: 0.2582825	test: 0.3928059	best: 0.3807404 (115)	total: 15.3s	remaining: 35.7s
800:	learn: 0.2319778	test: 0.3988169	best: 0.3807404 (115)	total: 21.2s	remaining: 31.8s
1000:	learn: 0.2084835	test: 0.4062443	best: 0.3807404 (115)	total: 24.5s	remaining: 24.5s
1200:	learn: 0.1892993	test: 0.4144622	best: 0.3807404 (115)	total: 27.7s	remaining: 18.4s
1400:	learn: 0.1726292	test: 0.4218708	best: 0.3807404 (115)	total: 31.3s	remaining: 13.4s
1600:	learn: 0.1581009	test: 0.4309876	best: 0.3807404 (115)	total: 36.7s	remaining: 9.16s
1800:	learn: 0.1451709	test: 0.4393404	best: 0.3807404 (115)	total: 39.9s	remaining: 4.41s
1999:	learn: 0.1341027	test: 0.4476455	best: 0.3807404 (115)	total: 43s	remaining: 0us

bestTest

In [39]:
pip install imbalanced-learn

In [41]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

# 데이터 준비
X = train_data.drop('y', axis=1)
y = train_data['y']

# 훈련 데이터와 검증 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE 적용
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# CatBoost 모델 초기화
catboost_model = CatBoostClassifier(
    iterations=2000, # 반복 횟수 증가
    learning_rate=0.05, # 학습률 감소
    depth=8, # 트리 깊이 조정
    loss_function='Logloss',
    verbose=200,
    scale_pos_weight=2 # 클래스 불균형 처리
)

# 모델 훈련
catboost_model.fit(X_train_smote, y_train_smote, eval_set=(X_val, y_val), use_best_model=True)

# 예측 및 평가
y_val_pred = catboost_model.predict(X_val)
f1 = f1_score(y_val, y_val_pred)
print(f'Validation F1 Score: {f1:.4f}')

0:	learn: 0.6333199	test: 0.6791233	best: 0.6791233 (0)	total: 103ms	remaining: 3m 25s
200:	learn: 0.1759175	test: 0.3933938	best: 0.3933858 (198)	total: 11s	remaining: 1m 38s
400:	learn: 0.1489704	test: 0.3905819	best: 0.3899189 (333)	total: 17.3s	remaining: 1m 8s
600:	learn: 0.1310187	test: 0.3945375	best: 0.3899189 (333)	total: 23.5s	remaining: 54.7s
800:	learn: 0.1171626	test: 0.3993614	best: 0.3899189 (333)	total: 31.2s	remaining: 46.6s
1000:	learn: 0.1059228	test: 0.4049913	best: 0.3899189 (333)	total: 36.8s	remaining: 36.7s
1200:	learn: 0.0970270	test: 0.4118627	best: 0.3899189 (333)	total: 45s	remaining: 30s
1400:	learn: 0.0890353	test: 0.4189767	best: 0.3899189 (333)	total: 50.9s	remaining: 21.7s
1600:	learn: 0.0815998	test: 0.4257176	best: 0.3899189 (333)	total: 59.2s	remaining: 14.7s
1800:	learn: 0.0753322	test: 0.4318352	best: 0.3899189 (333)	total: 1m 4s	remaining: 7.16s
1999:	learn: 0.0698698	test: 0.4397190	best: 0.3899189 (333)	total: 1m 13s	remaining: 0us

bestTest = 0

In [42]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

In [44]:
pip install catboost scikit-learn

In [45]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, f1_score

In [ ]:
# 데이터 준비
X = train_data.drop('y', axis=1)
y = train_data['y']

# 훈련 데이터와 검증 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 초기화
model = CatBoostClassifier(loss_function='Logloss', verbose=False)

# 탐색할 파라미터 그리드 설정
param_grid = {
    'iterations': [500, 1000, 1500],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5]
}

# F1 스코어를 계산하는 함수 생성
f1_scorer = make_scorer(f1_score)

# GridSearchCV 설정
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=f1_scorer, cv=3, verbose=2)

# 그리드 서치 실행
grid.fit(X_train, y_train)

# 최적의 파라미터와 최고 F1 스코어 출력
print("Best parameters found: ", grid.best_params_)
print("Best F1 score on training data: {:.4f}".format(grid.best_score_))

# 최적의 파라미터로 검증 데이터에 대해 평가
best_model = grid.best_estimator_
y_val_pred = best_model.predict(X_val)
val_f1_score = f1_score(y_val, y_val_pred)
print("Validation F1 Score: {:.4f}".format(val_f1_score))

Fitting 3 folds for each of 81 candidates, totalling 243 fits
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.01; total time=  12.9s
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.01; total time=   8.1s
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.01; total time=  10.6s
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.05; total time=  10.6s
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.05; total time=   4.7s
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.05; total time=   2.8s
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.1; total time=   2.8s
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.1; total time=   5.8s
[CV] END depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.1; total time=   2.8s
[CV] END depth=4, iterations=500, l2_leaf_reg=3, learning_rate=0.01; total time=   2.8s
[CV] END depth=4, iterations=500, l2_leaf_reg=3, learning_rat